### Import rdflib and create functions

* g is the ontology graph where the infered relations are added to
* Serialize is a function that transforms the ontology into a turle format
* Save is a function that can save the ontology as a turtle file
* Load is a function that loads the ontology into this file
* Shorten is a function that shows the prefix + subject, property or object    vb. kinship1:has_mother
* Relation shows only the subject, property or object

In [1]:
from rdflib import Graph, RDF, RDFS, Namespace, Literal, URIRef 
import pandas as pd

pd.set_option('display.max_rows', 999)


df = pd.DataFrame()
g = Graph()

KIN = Namespace('http://www.semanticweb.org/combots#')
g.bind('kinship1',KIN)

def serialize():
    print(g.serialize(format='turtle').decode("utf-8"))

def save(filename):
    with open(filename, 'wb') as f:
        g.serialize(f, format='turtle')
        
def load(filename):
    with open(filename, 'r') as f:
        g.load(f, format='turtle') 
    
def shorten(uri):
    if isinstance(uri, Literal):
        return uri    
    components = g.namespace_manager.compute_qname(uri)
    return '%s:%s'%(components[0], components[2])

def relation(uri):
    if isinstance(uri, Literal):
        return uri    
    components = g.namespace_manager.compute_qname(uri)
    return '%s'%(components[2])

## Create counter that counts the amount of newly created relations in this ontology.

In [2]:
new_relations_counter = 0

### Load in the ontology and print in turtle format

In [3]:
load('correct_output_ontology.ttl')
serialize()

@prefix kinship1: <http://www.semanticweb.org/combots#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

kinship1:Mrs._Bing kinship1:has_Son kinship1:Chandler .

kinship1:Rachel kinship1:has_Sister kinship1:Jill .

kinship1:Dina kinship1:has_Brother kinship1:Joey .

kinship1:Frank kinship1:has_Sister kinship1:Phoebe .

kinship1:Joey kinship1:has_Sister kinship1:Dina,
        kinship1:Tina .

kinship1:Monica kinship1:has_Brother kinship1:Ross .

kinship1:Ross kinship1:has_Father kinship1:Mr._Geller ;
    kinship1:has_Sister kinship1:Monica ;
    kinship1:has_Son kinship1:Ben .

kinship1:Ursula kinship1:has_Sister kinship1:Phoebe .

kinship1:Phoebe kinship1:has_Brother kinship1:Frank ;
    kinship1:has_Sister kinship1:Ursula .

[] a owl:Ontology .




### Create empty lists and values that have to be filled later.

In [4]:
parent  = []
parent2 = []
father  = []
mother  = []
child   = []
child2  = []
son     = []
daughter = []
grandchild = []
grandchild2 = []
grandparent = []
grandparent2 = []
inv1    = []
inv2    = []
disj1   = []
disj2   = []
couple  = []
couple2 = []
marriage = []
marriage2 = []
marriage3 = []
marriage4 = []
marriage5 = []
marriage6 = []
marriage7 = []
marriage8 = []
cousin1 = []
cousin2 = []
sibling1 = []
sibling2 = []

### Create a dataframe that later shows the inferred gender relations and dataframe that shows the inferred relations.

In [5]:
df = pd.DataFrame(columns = ["Relation", "Subject", "Property", "Object"])
df2 = pd.DataFrame(columns = ["Subject", "Property", "Object"])
df5 = pd.DataFrame(columns = ["Subject", "Property", "Object"])
# df8 = pd.DataFrame(columns = ["Subject", "Property", "Object"])
df_sibling =  pd.DataFrame(columns = ["Subject", "Object"])
# df25 = pd.DataFrame(columns = ["Subject", "Property", "Object"])

 ### Fill the lists with the right values and add gender to individuals
 The known relations will be added seperatly in lists and the genders of the individuals will be added to the ontology by determining what their relation is with other individuals. For example, if someone is a mother she will automatically also be a woman.

In [6]:
for s,p,o in g:
# PARENT RELATIONS
    if shorten(p) == "kinship1:has_parent" or shorten(p) == "kinship1:has_Father" or shorten(p) == "kinship1:has_Mother":
        parent.append(o)
        child.append(s) 
    if shorten(p) == "kinship1:has_Father":
        df = df.append({'Relation': "Father", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Mother":
        df = df.append({'Relation': "Mother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# GRANDPARENT RELATIONS
    if shorten(p) == "kinship1:has_Grandparent" or shorten(p) == "kinship1:has_Grandfather " or shorten(p) == "kinship1:has_Grandmother":
        grandparent.append(o)
        grandchild.append(s)    
    if shorten(p) == "kinship1:has_Grandfather":
        df = df.append({'Relation': "Grandfather", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Grandmother":
        df = df.append({'Relation': "Grandmother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# CHILD RELATIONS       
    if shorten(p) == "kinship1:has_Child" or shorten(p) == "kinship1:has_Son" or shorten(p) == "kinship1:has_Dauther": #typo in ontology daughter
        child2.append(o)
        parent2.append(s)    
    if shorten(p) == "kinship1:has_Son":
        df = df.append({'Relation': "Son", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Dauther":
        df = df.append({'Relation': "Daughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# GRANDCHILD RELATIONS       
    if shorten(p) == "kinship1:has_Grandchild" or shorten(p) == "kinship1:has_Grandson" or shorten(p) == "kinship1:has_Granddaughter": 
        grandchild2.append(o)
        grandparent2.append(s)   
    if shorten(p) == "kinship1:has_Grandson":
        df = df.append({'Relation': "Grandson", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Granddaughter":
        df = df.append({'Relation': "Granddaughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# SIBLING RELATIONS
    if shorten(p) == "kinship1:has_Sibling" or  shorten(p) == "kinship1:has_Sister" or shorten(p) == "kinship1:has_Brother": 
        sibling1.append(o)
        sibling2.append(s)
    if shorten(p) == "kinship1:has_Brother":
        df = df.append({'Relation': "Brother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Sister":
        df = df.append({'Relation': "Sister", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# COUSIN RELATIONS
    if shorten(p) == "kinship1:has_Cousin":
        cousin1.append(o)
        cousin2.append(s)

#RELATIONSHIPS 
    if shorten(p) == "kinship1:has_Husband" or shorten(p) == "kinship1:has_Wife":
        couple.append(o)
        couple2.append(s)
    if shorten(p) == "kinship1:has_Husband":
        marriage.append(o)
        marriage2.append(s)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
        df = df.append({'Relation': "Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Wife":
        marriage3.append(o)
        marriage4.append(s)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        df = df.append({'Relation': "Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_husband":
        marriage5.append(o)
        marriage6.append(s)
        g.add((o, RDFS.domain, KIN.man)) 
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_wife":
        marriage7.append(o)
        marriage8.append(s)
        g.add((o, RDFS.domain, KIN.woman))  
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)

### Print a table with the inferred gender relations that are added to the ontology

In [7]:
df = df.drop_duplicates().sort_values(by=['Relation'])
df.reset_index(drop=True)

,Relation,Subject,Property,Object
0,Brother,Frank,domain,man
1,Brother,Ross,domain,man
2,Brother,Joey,domain,man
3,Father,Mr._Geller,domain,man
4,Sister,Jill,domain,woman
5,Sister,Monica,domain,woman
6,Sister,Dina,domain,woman
7,Sister,Ursula,domain,woman
8,Sister,Phoebe,domain,woman
9,Sister,Tina,domain,woman


In [8]:
serialize() 

@prefix kinship1: <http://www.semanticweb.org/combots#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

kinship1:Mrs._Bing kinship1:has_Son kinship1:Chandler .

kinship1:Rachel kinship1:has_Sister kinship1:Jill .

kinship1:Ben rdfs:domain kinship1:man .

kinship1:Chandler rdfs:domain kinship1:man .

kinship1:Dina kinship1:has_Brother kinship1:Joey ;
    rdfs:domain kinship1:woman .

kinship1:Frank kinship1:has_Sister kinship1:Phoebe ;
    rdfs:domain kinship1:man .

kinship1:Jill rdfs:domain kinship1:woman .

kinship1:Joey kinship1:has_Sister kinship1:Dina,
        kinship1:Tina ;
    rdfs:domain kinship1:man .

kinship1:Monica kinship1:has_Brother kinship1:Ross ;
    rdfs:domain kinship1:woman .

kinship1:Mr._Geller rdfs:domain kinship1:man .

kinship1:Ross kinship1:has_Father kinship1:Mr._Geller ;
    kinship1:has_Sister kinship1:Monica ;
    kinship1:has_Son kinship1:Ben ;
    rdfs:domain kinship1:man .

kinship1:Tina rdfs:d

### Extract all the necessary properties

In [9]:
for s,p,o in g:
    if shorten(p) == "owl:inverseOf":
        inv1.append(o)
        inv2.append(s)
        if shorten(s) == "kinship1:has_Child":
            child_property = s 
        if shorten(o) == "kinship1:has_parent":
            parent_property = o
        if shorten(s) == "kinship1:has_Grandchild":
            grandchild_property = s
        if shorten(o) == "kinship1:has_Grandparent":
            grandparent_property = o
        
    if shorten(p) == "owl:propertyDisjointWith":
        disj1.append(o)
        disj2.append(s)
        if shorten(s) == "kinship1:has_Husband":
            husband_property = s
        if shorten(o) == "kinship1:has_Wife":
            wife_property = o
        if shorten(s) == "kinship1:has_Ex_husband":
            ex_husband_property = s
        if shorten(o) == "kinship1:has_Ex_wife":
            ex_wife_property = o
            
    if shorten(o) == "owl:SymmetricProperty":
        symprop = shorten(o)
        if shorten(s) ==  "kinship1:has_Cousin":
            cousin_property = s
        if shorten(s) == "kinship1:has_Sibling":
            sibling_property = s
            

    if shorten(o) == "owl:IrreflexiveProperty":
        if shorten(s) == "kinship1:has_Partner":
            partner_property = s
        if shorten(s) == "kinship1:has_Father":
            father_property = s
        if shorten(s) == "kinship1:has_Mother":
            mother_property = s
        if shorten(s) ==  "kinship1:has_Son"  :
            son_property = s
        if shorten(s) == "kinship1:has_Daughter":
            daughter_property = s
        if shorten(s) == "kinship1:has_Grandson":
            grandson_property = s
        if shorten(s) == "kinship1:has_Granddaughter":
            granddaughter_property = s
        if shorten(s) == "kinship1:has_Grandfather":
            grandfather_property = s
        if shorten(s) == "kinship1:has_Grandmother":
            grandmother_property = s
        if shorten(s) == "kinship1:has_Brother":
            brother_property = s
        if shorten(s) == "kinship1:has_Sister":
            sister_property = s
        if shorten(s) == "kinship1:has_twin":
            twin_property = s

Print the properties to show what they look like.

In [10]:
print("child_property", child_property)
print("parent_property", parent_property)
print("grandchild_property", grandchild_property)
print("grandparent_property", grandparent_property)
print("husband_property", husband_property)
print("wife_property", wife_property)
print("ex_husband_property", ex_husband_property)
print("ex_wife_property", ex_wife_property)
print("cousin_property", cousin_property)
print("partner_property", partner_property)
print("father_property", father_property)
print("mother_property", mother_property)
print("son_property", son_property)
print("daughter_property", daughter_property)
print("grandson_property", grandson_property)
print("granddaughter_property", granddaughter_property)
print("grandfather_property", grandfather_property)
print("grandmother_property", grandmother_property)
print("brother_property", brother_property)
print("sister_property", sister_property)
print("twin_property", twin_property)
print("sibling_property", sibling_property)


NameError: name 'child_property' is not defined

### Create functions to iterate through the lists and add the reversed relations with the right property to the ontology.
All the added relations are also added to a dataframe.

In [ ]:
def create_inferrence(new_subject, new_property, new_object, ignore_index=True):
    global df2
    global new_relations_counter
    for i in range(len### Create functions to iterate through the lists and add the reversed relations with the right property to the ontology.
All the added relations are also added to a dataframe.(new_subject)):
        g.add((new_subject[i], new_property, new_object[i]))
        new_relations_counter +=1
        data = [[ relation(new_subject[i]), relation(new_property), relation(new_object[i])]]
        df3 = pd.DataFrame(data, columns = ['Subject', 'Property', "Object"])
        df2 = df2.append(df3)

In [ ]:
create_inferrence(parent, child_property, child)
create_inferrence(child2, parent_property, parent2)
create_inferrence(grandparent, grandchild_property, grandchild)
create_inferrence(grandchild2, grandparent_property, grandparent2)

create_inferrence(couple, partner_property, couple2)
create_inferrence(marriage, wife_property, marriage2)
create_inferrence(marriage3, husband_property, marriage4)
create_inferrence(marriage5, ex_wife_property, marriage6)
create_inferrence(marriage7, ex_husband_property, marriage8)

create_inferrence(cousin1, cousin_property, cousin2)

# print(create_inferrence(sibling1, sibling_property, sibling2))

In [ ]:
df2.reset_index(drop=True)

In [ ]:
serialize() 

## Extract all the man and woman. And if someone participates in a relation that is not dependent on gender, they will be added to the appropriate list. 

In [ ]:
woman_list = []
man_list = []
has_parent_relation = []
has_child_relation = []
has_grandparent_relation = []
has_grandchild_relation = []
has_sibling_relation = []

for s,p,o in g:
    if shorten(o) ==  "kinship1:woman":
        woman_list.append(s)
    if shorten(o) ==  "kinship1:man":
        man_list.append(s)
    if shorten(p) == "kinship1:has_parent":# or shorten(p) == "kinship1:has_Father" or shorten(p) == "kinship1:has_Mother":
        has_parent_relation.append((o, s))
    if shorten(p) == "kinship1:has_Child":# or shorten(p) == "kinship1:has_Son" or shorten(p) == "kinship1:has_Dauther":
        has_child_relation.append((o, s))
    if shorten(p) == "kinship1:has_Grandparent":# or shorten(p) == "kinship1:has_Grandfather " or shorten(p) == "kinship1:has_Grandmother":
        has_grandparent_relation.append((o, s))
    if shorten(p) == "kinship1:has_Grandchild": #or shorten(p) == "kinship1:has_Grandson" or shorten(p) == "kinship1:has_Granddaughter":
        has_grandchild_relation.append((o, s))
    if shorten(p) == "kinship1:has_Sibling":
        has_sibling_relation.append((o,s))

## Add relations that are gender based.

In [ ]:
mothers = set(woman_list) & set([x[0] for x in has_parent_relation])
fathers = set(man_list) & set([x[0] for x in has_parent_relation])
daughters = set(woman_list) & set([x[0] for x in has_child_relation])
sons = set(man_list) & set([x[0] for x in has_child_relation])

sisters = set(woman_list) & set([x[0] for x in has_sibling_relation])
brothers = set(man_list) & set([x[0] for x in has_sibling_relation])

grandmothers = set(woman_list) & set([x[0] for x in has_grandparent_relation])
grandfathers = set(man_list) & set([x[0] for x in has_grandparent_relation])
granddaughters = set(woman_list) & set([x[0] for x in has_grandchild_relation])
grandsons = set(man_list) & set([x[0] for x in has_grandchild_relation])
grandmothers, grandfathers, granddaughters, grandsons, sisters, brothers

### Create functions to iterate through the lists and add the reversed relations with the right property to the ontology.
All the added relations are also added to a dataframe.

In [ ]:
def create_gender_based_inferrence(subject_list, new_property, relationship, ignore_index=True):
    global df5  
    global new_relations_counter
    for x in subject_list:
        if x[0] in relationship:
            g.add((x[1], new_property, x[0]))
            new_relations_counter +=1
            data = [[ relation(x[1]), relation(new_property), relation(x[0])]]
            df3 = pd.DataFrame(data, columns = ['Subject', 'Property', "Object"])
            df5 = df5.append(df3)

In [ ]:
create_gender_based_inferrence(has_parent_relation, mother_property, mothers)
create_gender_based_inferrence(has_parent_relation, father_property, fathers)
create_gender_based_inferrence(has_child_relation, daughter_property, daughters)
create_gender_based_inferrence(has_child_relation, son_property, sons)
create_gender_based_inferrence(has_grandparent_relation, grandmother_property, grandmothers)
create_gender_based_inferrence(has_grandparent_relation, grandfather_property, grandfathers)
create_gender_based_inferrence(has_grandchild_relation, granddaughter_property, granddaughters)
create_gender_based_inferrence(has_grandchild_relation, grandson_property, grandsons)
create_gender_based_inferrence(has_sibling_relation, sister_property, sisters)
create_gender_based_inferrence(has_sibling_relation, brother_property, brothers)

### Print a dataframe with all the inferred relations. 
These relations are all gender based, so father instead of parent ect.

In [ ]:
df5 = df5.sort_values(by=['Property'])
df5.reset_index(drop=True)

In [ ]:
print (g.serialize(format='turtle').decode("utf-8")) 

### Add another round of gender assigning, because there may be some individuals that can now be assigned a gender.

In [ ]:
for s,p,o in g:
# PARENT RELATIONS
    if shorten(p) == "kinship1:has_Father":
        df = df.append({'Relation': "Father", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Mother":
        df = df.append({'Relation': "Mother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# GRANDPARENT RELATIONS
    if shorten(p) == "kinship1:has_Grandfather":
        df = df.append({'Relation': "Grandfather", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Grandmother":
        df = df.append({'Relation': "Grandmother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        
# CHILD RELATIONS       
    if shorten(p) == "kinship1:has_Son":
        df = df.append({'Relation': "Son", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Dauther":
        df = df.append({'Relation': "Daughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# GRANDCHILD RELATIONS        
    if shorten(p) == "kinship1:has_Grandson":
        df = df.append({'Relation': "Grandson", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Granddaughter":
        df = df.append({'Relation': "Granddaughter", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1

# SIBLING RELATIONS
    if shorten(p) == "kinship1:has_Brother":
        df = df.append({'Relation': "Brother", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
    if shorten(p) == "kinship1:has_Sister":
        df = df.append({'Relation': "Sister", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
#RELATIONSHIPS 
    if shorten(p) == "kinship1:has_Husband":
        g.add((o, RDFS.domain, KIN.man))
        new_relations_counter +=1
        df = df.append({'Relation': "Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Wife":
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        df = df.append({'Relation': "Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_husband":
        g.add((o, RDFS.domain, KIN.man)) 
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Husband", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.man) }, ignore_index=True)
    if shorten(p) == "kinship1:has_Ex_wife":
        g.add((o, RDFS.domain, KIN.woman))
        new_relations_counter +=1
        df = df.append({'Relation': "Ex_Wife", 'Subject': relation(o), 'Property': relation(RDFS.domain),"Object":relation(KIN.woman) }, ignore_index=True)

In [ ]:
df = df.sort_values(by=['Subject'])
df = df.drop_duplicates()
df = df.drop(['Property', 'Object'], axis=1)
columns_titles = ["Subject","Relation"]
df = df.reindex(columns=columns_titles)
print("\n","A dataframe of all the individuals with the relations in which they participate.")
df.reset_index(drop=True)

### Add sibling relations

In [ ]:
sibling_infer_relation_list = []
sibling_infer_relation_list_show = []
name_list = []

for s,p,o in g:
    if shorten(p) == "kinship1:has_Sibling" or relation(p) == "has_Sister" or relation(p) == "has_Brother"  or relation(p) == "has_twin":
#         print(relation(s), relation(p), relation(o))
        sibling_infer_relation_list_show.append((relation(s), relation(o)))
        sibling_infer_relation_list.append((s, o))
        name_list.append(relation(s))
        name_list.append(relation(o))

Show all the sibling relations that exist at the moment.

In [ ]:
sibling_infer_relation_list_show

In [ ]:
name_data = pd.DataFrame(name_list, columns = ["Name"])
final_name_list = name_data.drop_duplicates().reset_index(drop=True)
print("\n", "These are all the individuals that participate in a sibling relation:")
final_name_list

### Put all the individuals that are siblings of each other in a set together and put those sets in a list. 

In [ ]:
def make_word_classes(synom):
    # List of classes
    word_classes = []
    for w1, w2 in synom:
        # Go through existing classes
        for wcls in word_classes:
            # If one of the words is already in the current class
            if w1 in wcls or w2 in wcls:
                # Add both words and continue to next pair of words
                wcls.add(w1)
                wcls.add(w2)
                break
        else:  # Note this else goes with the for loop, not the if block
            # If there was no matching class, add a new one
            word_classes.append({w1, w2})
    return word_classes

synom = sibling_infer_relation_list
show = sibling_infer_relation_list_show
word_classes = make_word_classes(synom)
# print(word_classes)
print("This is what it looks like right now:","\n", make_word_classes(show))

### Iterate through the list with the sets and add a sibling relation between all the individuals of a set except when the individuals are the same.

In [ ]:
vowels = word_classes
vowels_iter = iter(word_classes)

while True: 
    try: 
        for i in word_classes:                 #x en i printen hetzelfde
            x = next(vowels_iter)
            for p in i:                         # p is 1 persoon
                for y in i:                 #p blijft 1 ronde dezelfde persoon en y itereerd tijdens die ronde door de personen
                    if p == y:
#                         print("same")
                        pass
                    else:
#                         print(p, sibling_property,y)
                        g.add((p, sibling_property,y))
                        new_relations_counter +=1
          
    except StopIteration: 
        # if StopIteration is raised, 
        # break from loop 
        break


### Add the sibling relations to the dataframe to show all the inferences.

In [ ]:
woman_list = []
man_list = []
has_sibling_relation = []
has_twin_relation=[]

for s,p,o in g:
    if shorten(o) ==  "kinship1:woman":
        woman_list.append(s)
    if shorten(o) ==  "kinship1:man":
        man_list.append(s)
    if shorten(p) == "kinship1:has_Sibling":
        has_sibling_relation.append((o,s))
    if shorten(p) == "kinship1:has_twin":
        has_twin_relation.append((s,o))
sisters = set(woman_list) & set([x[0] for x in has_sibling_relation])
brothers = set(man_list) & set([x[0] for x in has_sibling_relation])
twin_sisters = set(woman_list) & set([x[0] for x in has_twin_relation])
twin_brothers = set(man_list) & set([x[0] for x in has_twin_relation])


# print(brothers,"\n", sisters, "\n",twin_brothers,"\n",twin_sisters)

create_gender_based_inferrence(has_sibling_relation, sister_property, sisters)
create_gender_based_inferrence(has_sibling_relation, brother_property, brothers)
create_gender_based_inferrence(has_twin_relation, twin_property, twin_brothers)
create_gender_based_inferrence(has_twin_relation, twin_property, twin_sisters)

In [ ]:
df5 = df5.drop_duplicates().sort_values(by=['Property'])
df5.reset_index(drop=True)

# Show final ontology

In [11]:
serialize()

@prefix kinship1: <http://www.semanticweb.org/combots#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

kinship1:Mrs._Bing kinship1:has_Son kinship1:Chandler .

kinship1:Rachel kinship1:has_Sister kinship1:Jill .

kinship1:Ben rdfs:domain kinship1:man .

kinship1:Chandler rdfs:domain kinship1:man .

kinship1:Dina kinship1:has_Brother kinship1:Joey ;
    rdfs:domain kinship1:woman .

kinship1:Frank kinship1:has_Sister kinship1:Phoebe ;
    rdfs:domain kinship1:man .

kinship1:Jill rdfs:domain kinship1:woman .

kinship1:Joey kinship1:has_Sister kinship1:Dina,
        kinship1:Tina ;
    rdfs:domain kinship1:man .

kinship1:Monica kinship1:has_Brother kinship1:Ross ;
    rdfs:domain kinship1:woman .

kinship1:Mr._Geller rdfs:domain kinship1:man .

kinship1:Ross kinship1:has_Father kinship1:Mr._Geller ;
    kinship1:has_Sister kinship1:Monica ;
    kinship1:has_Son kinship1:Ben ;
    rdfs:domain kinship1:man .

kinship1:Tina rdfs:d

In [12]:
save("new_correctdata1_ontology.ttl")

# Show the result of the counter of the newly added relations to the ontology


In [13]:
new_relations_counter

13